In [1]:
import pandas as pd

Read the data

In [2]:
data = pd.read_csv('train.csv')
challenge = pd.read_csv('test.csv')

In [3]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Combine dataset to process them

In [6]:
combined = pd.concat([data, challenge], axis=0)

Drop un-necessary variables 

In [7]:
combined = combined.drop(columns=['Name', 'Ticket'])

Generate dummy variables

In [8]:
combined = pd.get_dummies(combined, columns=['Sex', 'Cabin','Embarked'])

Split into train/test and challenge datasets

In [9]:
data = combined.loc[~combined.Survived.isna()]
challenge = combined.loc[combined.Survived.isna()]

Fill NaN with column medians

In [10]:
data = data.fillna(data.median())
challenge = challenge.fillna(challenge.median())

Split into test and train

In [11]:
split_index = int(data.shape[0] * 0.8)

train = data.iloc[:split_index]
test = data.iloc[split_index:]

In [12]:
X = train.drop(columns=['PassengerId', 'Survived'])
y = train[['Survived']]

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
clf = RandomForestClassifier(random_state=1)
clf.fit(X, y)

/var/folders/9b/x1yljdln1dv4bsg1w8xmqlpc0000gn/T/ipykernel_23068/1125411724.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)


RandomForestClassifier(random_state=1)

In [15]:
X_test = test.drop(columns=['PassengerId', 'Survived'])
y_test = test[['Survived']]

In [16]:
clf.score(X_test, y_test)

0.8212290502793296

In [17]:
X_challenge = challenge.drop(columns=['PassengerId', 'Survived'])

Generate submission

In [18]:
submission = pd.concat([challenge[['PassengerId']], pd.DataFrame(clf.predict(X_challenge))], axis=1)
submission.columns = ['PassengerId', 'Survived']
submission.Survived = submission.Survived.astype(int)

Save submission

In [19]:
submission.to_csv('submission.csv', index=False)